<a href="https://colab.research.google.com/github/Pushpit07/Tech-Rush_AI-Ranking/blob/main/Tech_Rush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement 2

Sears is a pioneer in the retail industry and operates in a multitude of domains, selling millions of products on a daily basis and continuously adding thousands of new products to its product line.
In order to effectively evaluate the performance of retail stores, it is essential to have a consistent analysis system in place factoring the local market and competition.

Given a list of store location data, build a system that will generate a ranking of the locations based on the following -

   -- Distance from nearest Walmart, Costco, Target, and Home Depot stores. Each of these distances will have its own weightage that has to be accounted for in the ranking algorithm.
   
   -- Population density in and around the location.
   
   -- Economy around the location

The weightage of these parameters needs to be configurable in the system. 

## Installation & Imports

In [12]:
%%capture
!pip install uszipcode

import pandas as pd
import numpy as np
import requests

try:
  from uszipcode.search import SearchEngine
  search = SearchEngine()
except:
  pass

Exception ignored in: <function SearchEngine.__del__ at 0x7fb8c1972430>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 196, in __del__
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 202, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1811, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1853, in _close_impl
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 923, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2416, in close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2639, in _do_close
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2625, in _close_impl
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2617, in _connection_rollback_impl
  File "/usr/lo

## Loading the dataset

In [13]:
# Load store location data
data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/locn_data.csv")

df = data[["locn_nbr", "locn_cty", "locn_st_cd", "zip_cd", "zip_pls_4", "locn_addr"]]
df.head()

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST


In [14]:
df.count()

locn_nbr      99
locn_cty      99
locn_st_cd    99
zip_cd        99
zip_pls_4     99
locn_addr     99
dtype: int64

## Adding coordinates, population density, and economy data for locations given in the dataset

In [15]:
START_INDEX = 0
END_INDEX = 100

def get_address(address):
    address = address.replace("#", "")
    url = 'https://nominatim.openstreetmap.org/search.php?q=' + address.replace(" ", "+") + "&format=json"
    response = requests.get(url).json()
    return response

for index, row in df.iterrows():
     if(index >= START_INDEX and index < END_INDEX):
        try:
          address = row['locn_addr'].strip() + " " + row['locn_cty'].strip() + " "  + row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
          response = get_address(address)

          if not response:
            address = row['locn_cty'].strip() + " "  + row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
            response = get_address(address)

          if not response:
            address = row['locn_addr'].strip() + " "  + row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
            response = get_address(address)

          if not response:
            address = row['locn_st_cd'].strip() + " " + str(row['zip_cd'])
            response = get_address(address)

          if(response):
            df.at[index, 'latitude'] = response[0]['lat']
            df.at[index, 'longitude'] = response[0]['lon']
            df.at[index, 'display_name'] = response[0]['display_name']
          
          zipcode = search.by_zipcode(row['zip_cd'])
          # Checking for non std postal codes
          if not zipcode.population:
              # Checking for non std zipcodes like postal boxes
              res = search.by_city_and_state(city=zipcode.major_city, state=zipcode.state)
              if (len(res)) > 0:
                  zipcode = res[0]

          df.at[index, 'population_density'] = zipcode.population_density
          df.at[index, 'median_household_income'] =  zipcode.median_household_income

        except Exception as e:
            print("An exception occurred:", e)
            continue

df['normalised_population_density'] = (((df['population_density'] - df['population_density'].min()) / (df['population_density'].max() - df['population_density'].min())) * 100)
df['normalised_economy'] = (((df['median_household_income'] - df['median_household_income'].min()) / (df['median_household_income'].max() - df['median_household_income'].min())) * 100)

df.head()

An exception occurred: 'NoneType' object has no attribute 'population'
An exception occurred: 'NoneType' object has no attribute 'population'


,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr,latitude,longitude,display_name,population_density,median_household_income,normalised_population_density,normalised_economy
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W,31.7784751,-106.38271330797117,"Cinemark, 8401, Gateway Boulevard West, El Pas...",2402.0,44217.0,5.749284,15.005663
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD,37.9647421,-122.0603194,"Sears Auto Center, 1001, Sunvalley Boulevard, ...",3688.0,47313.0,8.925220,17.267911
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN,33.9457006,-117.9995233,"Whittwood Lane, Whittwood Town Center, Friendl...",6136.0,90477.0,14.970858,48.807862
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL,47.4627356,-122.2559156,"Tukwila, King County, Washington, United States",3062.0,40815.0,7.379235,12.519820
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST,42.22089312121212,-71.02840827272726,"250, Granite Street, Braintree, Norfolk County...",2600.0,81844.0,6.238269,42.499726


In [17]:
df.isnull().sum()

locn_nbr                          0
locn_cty                          0
locn_st_cd                        0
zip_cd                            0
zip_pls_4                         0
locn_addr                         0
latitude                          0
longitude                         0
display_name                      0
population_density               12
median_household_income          12
normalised_population_density    12
normalised_economy               12
dtype: int64

In [18]:
df = df.dropna()
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Removing outliers that are outside the area of the United States
df = df[df['latitude'] > 15]
# Removing outliers
df = df[df['latitude'] < 70]
# Removing outliers
df = df[df['longitude'] < -66]
# Removing outliers
df = df[df['longitude'] > -170]

<ipython-input-18-c98008fd10f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = df['latitude'].astype(float)
<ipython-input-18-c98008fd10f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = df['longitude'].astype(float)


In [19]:
# The haversine formula is used to  calculate the distance between two geographic coordinates on the earth. 
# This formula calculates the great-circle distance between two points on a sphere given their longitudes and latitudes.

import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    # Calculate the haversine distance
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of the earth in kilometers
    return c * r

## Calculating distance from nearest Walmart

In [20]:
# Load walmart stores location data
walmart_stores_data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/walmartstoreloaction.csv")

for index, row in df.iterrows():
    distances_to_walmarts = []
    if(index >= START_INDEX and index < END_INDEX):
        for idx, walmart_store in walmart_stores_data.iterrows():
          distances_to_walmarts.append(haversine_distance(float(row['latitude']), float(row['longitude']), float(walmart_store['latitude']), float(walmart_store['longitude'])))
        min_dist = min(distances_to_walmarts)
        df.at[index, 'distance_from_nearest_walmart'] = min_dist

## Calculating distance from nearest Costco

In [21]:
# Load costco stores location data
costco_stores_data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/costco_store.csv")

for index, row in df.iterrows():
    distances_to_costcos = []
    if(index >= START_INDEX and index < END_INDEX):
        for idx, costco_store in costco_stores_data.iterrows():
          distances_to_costcos.append(haversine_distance(float(row['latitude']), float(row['longitude']), float(costco_store['latitude']), float(costco_store['longitude'])))
        min_dist = min(distances_to_costcos)
        df.at[index, 'distance_from_nearest_costco'] = min_dist

## Calculating distance from nearest Target

In [24]:
# Load target stores location data
target_stores_data = pd.read_csv("https://raw.githubusercontent.com/Pushpit07/Tech-Rush_AI-Ranking/main/target.csv", encoding='latin-1')

for index, row in df.iterrows():
    distances_to_targets = []
    if (index >= START_INDEX and index < END_INDEX):
        for idx, target_store in target_stores_data.iterrows():
            distances_to_targets.append(haversine_distance(float(row['latitude']), float(row['longitude']), float(target_store['Address.Latitude']), float(target_store['Address.Longitude'])))
        min_dist = min(distances_to_targets)
        df.at[index, 'distance_from_nearest_target'] = min_dist

In [25]:
df['normalised_distance_from_nearest_walmart'] = (((df['distance_from_nearest_walmart'] - df['distance_from_nearest_walmart'].min()) / (df['distance_from_nearest_walmart'].max() - df['distance_from_nearest_walmart'].min())) * 100)
df['normalised_distance_from_nearest_costco'] = (((df['distance_from_nearest_costco'] - df['distance_from_nearest_costco'].min()) / (df['distance_from_nearest_costco'].max() - df['distance_from_nearest_costco'].min())) * 100)
df['normalised_distance_from_nearest_target'] = (((df['distance_from_nearest_target'] - df['distance_from_nearest_target'].min()) / (df['distance_from_nearest_target'].max() - df['distance_from_nearest_target'].min())) * 100)

In [26]:
# Weightage for each parameter
DISTANCE_WEIGHT = 0.4
POPULATION_DENSITY_WEIGHT = 0.3
ECONOMY_WEIGHT = 0.3

NEAREST_WALMART_DISTANCE_WEIGHT = 0.5
NEAREST_COSTCO_DISTANCE_WEIGHT = 0.3
NEAREST_TARGET_DISTANCE_WEIGHT = 0.2

NUMBER_OF_TOP_RESULTS = 5

In [27]:
df.head()

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr,latitude,longitude,display_name,population_density,median_household_income,normalised_population_density,normalised_economy,distance_from_nearest_walmart,distance_from_nearest_costco,distance_from_nearest_target,normalised_distance_from_nearest_walmart,normalised_distance_from_nearest_costco,normalised_distance_from_nearest_target
0,1317,EL PASO,TX,79925,5600,8401 GATEWAY BLVD W,31.778475,-106.382713,"Cinemark, 8401, Gateway Boulevard West, El Pas...",2402.0,44217.0,5.749284,15.005663,0.213845,2.716829,3.069566,0.048442,1.237778,2.164797
1,1368,CONCORD,CA,94520,5802,1001 SUNVALLEY BLVD,37.964742,-122.060319,"Sears Auto Center, 1001, Sunvalley Boulevard, ...",3688.0,47313.0,8.925220,17.267911,5.597910,2.336812,1.512346,4.267011,0.993956,0.975168
2,1149,WHITTIER,CA,90603,2300,15600 WHITTWOOD LN,33.945701,-117.999523,"Whittwood Lane, Whittwood Town Center, Friendl...",6136.0,90477.0,14.970858,48.807862,3.274059,3.064878,0.439408,2.446207,1.461089,0.155504
3,1139,TUKWILA,WA,98188,2811,301 SOUTHCENTER MALL,47.462736,-122.255916,"Tukwila, King County, Washington, United States",3062.0,40815.0,7.379235,12.519820,2.819904,2.042249,0.847777,2.090364,0.804961,0.467475
4,1283,BRAINTREE,MA,2184,2803,250 GRANITE ST,42.220893,-71.028408,"250, Granite Street, Braintree, Norfolk County...",2600.0,81844.0,6.238269,42.499726,3.862893,10.097019,0.543708,2.907576,5.972978,0.235183


## Generate the final score for each location

In [28]:
for index, row in df.iterrows():
        if (index >= START_INDEX and index < END_INDEX):
            try:
                # Assigning a score based on greatest population density, higher economy, and low distances from Walmart, Costco, and Target
                score_of_distance_from_stores = (df.at[index, 'normalised_distance_from_nearest_walmart'] * NEAREST_WALMART_DISTANCE_WEIGHT + df.at[index, 'normalised_distance_from_nearest_costco'] * NEAREST_COSTCO_DISTANCE_WEIGHT + df.at[index, 'normalised_distance_from_nearest_target'] * NEAREST_TARGET_DISTANCE_WEIGHT) * DISTANCE_WEIGHT
                df.at[index, 'score'] = df.at[index, 'normalised_population_density'] * POPULATION_DENSITY_WEIGHT + df.at[index, 'normalised_economy'] * ECONOMY_WEIGHT - score_of_distance_from_stores

            except Exception as e:
                print("An exception occurred", e)
                continue

df['normalised_score'] = (((df['score'] - df['score'].min()) / (df['score'].max() - df['score'].min())) * 100)
df['rank'] = df['normalised_score'].rank(ascending=False)

In [29]:
top_df = df.nsmallest(NUMBER_OF_TOP_RESULTS, "rank")
top_df

,locn_nbr,locn_cty,locn_st_cd,zip_cd,zip_pls_4,locn_addr,latitude,longitude,display_name,population_density,...,normalised_economy,distance_from_nearest_walmart,distance_from_nearest_costco,distance_from_nearest_target,normalised_distance_from_nearest_walmart,normalised_distance_from_nearest_costco,normalised_distance_from_nearest_target,score,normalised_score,rank
7,1044,JERSEY CITY,NJ,7310,1601,50 MALL DR WEST,40.729224,-74.038366,"Mall Drive West, Newport Centre Mall, Jersey C...",20880.0,...,80.344160,7.099959,8.414460,0.411637,5.443909,4.893432,0.134288,37.831408,100.000000,1.0
42,9451,BROOKLYN,NY,11214,0,ATLANTIC AVE AND FLATBUSH,40.652601,-73.949721,"Brooklyn, Kings County, City of New York, New ...",40566.0,...,14.407219,13.269277,4.873836,2.356447,10.277747,2.621734,1.620014,31.822407,90.181836,2.5
75,9468,BROOKLYN,NY,11214,0,FLATBUSH AND NORSTRAND AV,40.652601,-73.949721,"Brooklyn, Kings County, City of New York, New ...",40566.0,...,14.407219,13.269277,4.873836,2.356447,10.277747,2.621734,1.620014,31.822407,90.181836,2.5
74,9452,WASHINGTON,DC,20222,0,RI AVE AND BRENTWOOD ROAD,38.895037,-77.036543,"Washington, District of Columbia, United States",17689.0,...,40.310548,2.212145,4.085831,3.267991,1.614167,2.116142,2.316383,24.381809,78.024571,4.0
18,5205,BRIDGEHAMPTON,NY,11976,0,2044 MONTAUK HWY,40.931947,-72.318064,"2044, Montauk Highway, Bridgehampton, Town of ...",128.0,...,100.000000,33.077211,32.406772,30.351125,25.797831,20.287126,23.006383,20.605476,71.854383,5.0
